# Signal Spatial Diversity Analysis

In [1]:
from scipy import signal 
import numpy as np

import timeit

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from bokeh.models.annotations import Title
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.palettes import Category10,viridis,Spectral11
from bokeh.models import ColumnDataSource, Range1d, BoxAnnotation, HoverTool, LogColorMapper, LogTicker, ColorBar
output_notebook()

samplefreq = 20*10**3 #Hz
T = 4 #1/(BW*10**3) # time (seconds)
num_samples = int(T*samplefreq)
rayDistance = [] #for physical ray modelling

# Types of Input Signal
def CWsinewave(freq1,T1,num_samples1): #Signal is a pure tonal CW(Continuous Wave) signal, Hence it is assumed to be a very narrowband signal
    freq1 = freq1*10**3 #Hz
    t1 = np.linspace(0,T1,num_samples1,endpoint = False)
    #Tx1 = np.abs(np.exp(2j * np.pi * freq1 * t1)) #+ complex(np.sin(2 * np.pi * freq1 * t1))
    Tx1 = np.sin(2*np.pi*freq1*t1)
    return Tx1

# Functions for RayModelling
def findDistance(h,r,d1,d2,s,b):
	global distance
	if s>b or s==b:
		distance = np.sqrt(r**2+((2*b*h)+d1-((-1)**(s-b))*d2)**2)
	if b>s:# or s==b:
		distance = np.sqrt(r**2+((2*b*h)-d1-((-1)**(s-b))*d2)**2)
	rayDistance.append(distance)
	return distance

def findlss(r):
	lss = 0
	lss =  1.0/r
	return lss

def findAngle(h,r,d1,d2,s,b):
	angle = 0
	if s>b:
		angle = 180*np.arctan2(r,((2*b*h)+d1-((-1)**(s-b))*d2))/np.pi
	if b>s:
		angle = 180*np.arctan2(r,((2*b*h)-d1+((-1)**(s-b))*d2))/np.pi
	if s==0 and b==0:
		angle = 0
	return angle

def findlb (angle,s,b):
	lb = 0
	m = (1500.0/1023)
	n = (1539.0/1650)
	lb = np.absolute((m*np.cos(np.pi*angle/180.0)-np.sqrt((n**2-(np.sin(np.pi*angle/180.0))**2)+0j))/(m*np.cos(np.pi*angle/180.0)+np.sqrt((n**2-(np.sin(np.pi*angle/180.0))**2)+0j)))
	return lb

def findla (freq, distance):
	la = 0
	salinity = 35 #value search on internet based on salinity in singapore
	ft = 21.9*10**(6-1520/273+26) #divided by Ocean Temp which is 26 celsius for shallow water about 70m deep
	la = np.exp(-0.998*distance*(((salinity*(2.34*10**-6)*ft*freq)/(ft**2+freq**2))+(((3.38*10**-6)*freq**2)/(ft))))
	return la

def findSamples(samples,distance1, distance2):
	tao = (distance1 - distance2)/1500 #speed of sound underwater m/s
	randomTao = tao + np.random.normal(tao)#adding tap migration using gaussian RV
	samples += round(tao*samplefreq,0) #time x sampling frequency = number of samples !!!! DOES NOT MAKE SENSE?
#	print("value of d", d)
	samples.append(int(d))
	return randomTao, samples

def ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d2,fc):
    """
    This function aims to provide users to create an impulse response based on Ray Tracing modelling. It employs 

    """
    
    global d,tao
    distanceD = findDistance(h,r,d1,d2,0,0)
#rayModelling
    Bja = []
    samples = []
    d = 0
    tao = 0
    s,b,e = [0,0,0]
    for j in range (int(eigen/3)+1):
        for i in range (3):
            if eigen == e:
                break
            e += 1
            findDistance(h,r,d1,d2,s,b)
            Bj = findlss(r)*findla(fc, distance)*(findlb(findAngle(h,r,d1,d2,s,b),s,b)**b)*1 # Direct Delta function
            if s%2!=0:
                Bj = -Bj
            Bja.append(Bj)
            findSamples(samples,distance, distanceD)

# to change the s and b values
            if i == 0 or i == 2:
                s+= 1
            elif i == 1:
                s-= 1
                b+= 1     
        
    Hn = np.zeros(int(d)+100)
    j,k = [0,0]
    while j <= int(d):
        if j == samples[k]:
            Hn[j] = Bja[k]
            k += 1
            if k == len(samples):
                break
            if samples[k]==samples[k-1]:
                Hn[j] = Bja[k]+Bja[k-1]
                k+=1
        j += 1
    
    return Hn

#RECEIVED Signal
def convolutionForReceivedSignal (Tx, Hn,**kwargs):
    Rx = []
    Rx = signal.fftconvolve(Tx,Hn,mode = "full")
    max_value = 'no max value, because it would not make sense'
    
    Rx_out = kwargs.get('Rx_out', True)
    Noise =kwargs.get('Noise', None)
    Filter = kwargs.get('Filter', None)
    
    # Noise Added
    if Noise == 'white':
        mean = 0
        std = 0.01
        white_noise = np.random.normal(mean, std, size=len(Rx))
        Rx += white_noise
    
    if Filter == True:
        max_value = filteredReceivedAmplitude(Rx)

    if Rx_out == False:
        return max_value
    else:
        return Rx, max_value

def filteredReceivedAmplitude(Rx):
    #Rx_new = Rx[int(len(Rx)/2):int(len(Rx)-(len(Rx)/2))]
    Rx_new = Rx[3000:4000]
    max_value = max(Rx_new)

    return max_value

def channelFrequencyResponse(Hn,num_samples,T,freq_list):
    max_new_list = []
   
    for freq in freq_list:
        Tx = CWsinewave(freq,T,num_samples)
        Max = convolutionForReceivedSignal(Tx,Hn,Rx_out = False, Filter = True)
        max_new_list.append(Max)

   
    return max_new_list

def FrequencyResponse(H,R,D1,D2_list,Eigen,samplefreq, fc, bw, **kwargs):
    """
    This function is able to compare different frequency responses of a given channel characterstis
    from varying the vertical position of the receiver, in this case D2.
    
    """
    Type = kwargs.get('Type', 'multi')
    
    lowfreq = fc-(bw/2.0)
    highfreq = fc+(bw/2.0)
    
    Hn_list = []
    Hn_len = []
    freq_final_list = []
    max_final_new_list = []
    
    if Type == 'multi':
        for D2 in D2_list:
            Hn = ImpulseResponse(Tx,Eigen,samplefreq,H,R,D1,D2)
            #signalSize = round(Hn.size/2000)*2000
            signalSize = Hn.size
            freq_l = np.fft.fftfreq(signalSize,1000/samplefreq)
            freq_list=[freq for freq in freq_l if freq>=lowfreq and freq<=highfreq]
            max_new_list = np.abs(np.fft.rfft(Hn))
            max_new_list = max_new_list[int(freq_list[0]):len(freq_list)+int(freq_list[0])]#-int(freq_list[0])]


            freq_list = freq_list[:(len(max_new_list))]
            freq_final_list.append(freq_list)
            max_final_new_list.append(max_new_list)
            
        return freq_final_list,max_final_new_list,freq_list
    
    if Type == 'duo':
        Hn1 = ImpulseResponse(Tx,Eigen,samplefreq,H,R,D1,D2_list[0])
        signalSize = round(Hn1.size/2000)*2000
        freq_l = np.fft.fftfreq(signalSize,1000/samplefreq)
        freq_list1=[freq for freq in freq_l if freq>=lowfreq and freq<=highfreq]
        max_new_list = np.abs(np.fft.rfft(Hn1))
        max_new_list1 = max_new_list[int(freq_list1[0]):len(freq_list1)+int(freq_list1[0])]#-int(freq_list[0])]
        mean1 = np.mean(max_new_list1)
        normFact1 = np.sqrt((sum((a-mean1)**2 for a in max_new_list1)))
        A = [(max_value-mean1)/normFact1 for max_value in max_new_list1]

        freq_list1 = freq_list1[:(len(A))]
        freq_final_list.append(freq_list1)
        max_final_new_list.append(A)


        Hn2 = ImpulseResponse(Tx,Eigen,samplefreq,H,R,D1,D2_list[1])
        signalSize = round(Hn2.size/2000)*2000
        freq_l = np.fft.fftfreq(signalSize,1000/samplefreq)
        freq_list2=[freq for freq in freq_l if freq>=lowfreq and freq<=highfreq]
        max_new_list = np.abs(np.fft.rfft(Hn2))
        max_new_list2 = max_new_list[int(freq_list2[0]):len(freq_list2)+int(freq_list2[0])]#-int(freq_list[0])]
        mean2 = np.mean(max_new_list2)
        normFact2 = np.sqrt((sum((b-mean2)**2 for b in max_new_list2)))
        B = [(max_value-mean2)/normFact2 for max_value in max_new_list2]
        
        freq_list2 = freq_list2[:(len(max_new_list2))]
        freq_final_list.append(freq_list2)
        max_final_new_list.append(B)
        
        CMNV = sum(a*b for a, b in zip(A, B))#/len(A)
        #CMNV = sum(a*b for a, b in zip(A, B))/(np.sqrt((sum(a**2 for a in A)*sum(b**2 for b in B))))
        CMNS = sum(a**2 for a in A)#/len(A)
        #CMNS = sum(a**2 for a in A)/(np.sqrt((sum(a**2 for a in A)*sum(a**2 for a in A))))
        return freq_final_list,max_final_new_list,CMNV, CMNS

def CorrDiversity(Tx,h,r,d1,d21,d22_list,eigen,samplefreq, fc, bw, **kwargs):
    tic=timeit.default_timer()
    Type = kwargs.get('Type', 'multi')
    
    lowfreq = fc-(bw/2)
    highfreq = fc+(bw/2)
    
    CMNL = []
    
    Hn1 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d21)
    signalSize1 = Hn1.size
    print("signal Size 1 before round off: ", signalSize1)
    signalSize1 = round(Hn1.size/2000)*2000
    print("signal Size 1 after round off: ", signalSize1)
    print("samplefreq: ", samplefreq)
    freq_l = np.fft.fftfreq(signalSize1,1000/samplefreq)
    freq_list_1=[freq for freq in freq_l if freq>=lowfreq and freq<=highfreq]
    #print(freq_list_1)
    
    max_new_list_1 = np.abs(np.fft.fft(Hn1)) #freq_list == samplefreq
    max_new_list_1 = max_new_list_1[int(freq_list_1[0]):len(freq_list_1)+int(freq_list_1[0])]
    mean1 = np.mean(max_new_list_1)
    A = [max_value-mean1 for max_value in max_new_list_1]
    
    if Type == 'multi':
        for d22 in d22_list:
            Hn2 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d22)
            #signalSize2 = Hn2.size
            signalSize2 = round(Hn2.size/2000)*2000
            freq_l = np.fft.fftfreq(signalSize2,1000/samplefreq)
            freq_list_2=[freq for freq in freq_l if freq>=lowfreq and freq<=highfreq]
            
            max_new_list_2 = np.abs(np.fft.fft(Hn2))
            max_new_list_2 = max_new_list_2[int(freq_list_2[0]):len(freq_list_2)+int(freq_list_2[0])]
            mean2 = np.mean(max_new_list_2)
            B = [max_value-mean2 for max_value in max_new_list_2]
            CMNL.append(sum(a*b for a, b in zip(A, B))/(np.sqrt((sum(a**2 for a in A)*sum(b**2 for b in B)))))
            normalizedCMNL = [CMNV/max(CMNL) for CMNV in CMNL]

        toc=timeit.default_timer()
        #print("time taken: ", toc-tic, 's') 
        return normalizedCMNL       
    
    if Type == 'single':        
        Hn2 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d22_list) ##not really a list (it's just one value of d2)
        #signalSize2 = Hn2.size
        signalSize2 = round(Hn2.size/2000)*2000
        freq_l = np.fft.fftfreq(signalSize2,1000/samplefreq)
        freq_list_2=[freq for freq in freq_l if freq>=lowfreq and freq<=highfreq]
        
        max_new_list_2 = np.abs(np.fft.fft(Hn2))
        max_new_list_2 = max_new_list_2[int(freq_list_2[0]):len(freq_list_2)+int(freq_list_2[0])]
        mean2 = np.mean(max_new_list_2)
        B = [max_value-mean2 for max_value in max_new_list_2]


        CMNV = (sum(a*b for a, b in zip(A, B))/(np.sqrt((sum(a**2 for a in A)*sum(b**2 for b in B))))) 
    
        toc=timeit.default_timer()
        #print("time taken: ", toc-tic, 's') 
        return CMNV
    
def avgXCorrForDeltas(Tx, H, R, D1, D2min, D2max, Eigen, deltaMin, deltaMax, thres, fc, bw, **kwargs):
    length = kwargs.get('Len', 'valid')
    
    delta_list = np.linspace(deltaMin,deltaMax,80)#(deltaMax-deltaMin)*25)
    d21_list = np.linspace(D2min,D2max,10) #for averaging purpose
    
    multiAvgCMNV = []
    
    if length == 'full':
        for j in range(len(delta_list)):
            forAverageCMNV_list = []
            for i in range(len(d21_list)):
                d22 = 0
                d22 = d21_list[i]+delta_list[j]
                CMNV = CorrDiversity(Tx,H,R,D1,d21_list[i],d22,Eigen,samplefreq,fc,bw,Type = 'single')
                forAverageCMNV_list.append(CMNV)
            avgCMNV = np.mean(forAverageCMNV_list)
            multiAvgCMNV.append(avgCMNV)
            normMultiAvgCMNV = [value/max(multiAvgCMNV) for value in multiAvgCMNV]
            if normMultiAvgCMNV[j] > thres:
                imptDelta = delta_list[j]
    
    if length == 'valid':
        for j in range(len(delta_list)):
            forAverageCMNV_list = []
            for i in range(len(d21_list)):
                d22 = 0
                #print("delta [{0}]".format(j), delta_list[j])
                #print("d21 [{0}]".format(i), d21_list[i])
                d22 = d21_list[i]+delta_list[j]
                #print("d22 = ", d22)
                CMNV = CorrDiversity(Tx,H,R,D1,d21_list[i],d22,Eigen,samplefreq,fc,bw,Type = 'single')
                forAverageCMNV_list.append(CMNV)
                #print("CMNV_list for avg: ", forAverageCMNV_list)
            avgCMNV = np.mean(forAverageCMNV_list)
            multiAvgCMNV.append(avgCMNV)
            #print("multiAvgCMNV: ",multiAvgCMNV)
            #normMultiAvgCMNV = [value/max(multiAvgCMNV) for value in multiAvgCMNV]
            normMultiAvgCMNV = [value for value in multiAvgCMNV]
            #print("normMultiAvgCMNV: ", normMultiAvgCMNV)
            if normMultiAvgCMNV[j] < thres:
                imptDelta = delta_list[j]
                break
            
        delta_list = delta_list[:len(normMultiAvgCMNV)]
        #print("multiAvgCMNV:", multiAvgCMNV)
    
    return delta_list, normMultiAvgCMNV, imptDelta
    
    
    
    


Loading BokehJS ...

# Channel Analysis Example

##### Input Signal

The input signal given in this example is a pure sinusoid that goes for 10 seconds.(10 seconds is used to simulate an infinitely long signal that will result in a super narrow band frequency in the frequency domain). A more accurate representation of a pure tonal signal will actually be an complex exponential that represents cos(2pixfcxt)+jsin(2pixfcxt). However, due to limitation in plotting using bokeh, a pure sine wave is employed. (Bokeh, or maybe any other plotting library for that matter, is unable to plot complex figures)

In [14]:
tic=timeit.default_timer()
fc = 1.35 #kHz
Tx = CWsinewave(fc,T,num_samples)

S = figure(title = "Continuous Sine Wave Input Signal (pure tonal)", x_axis_label = "Time(s)", y_axis_label = "Ampltidue",plot_width=500, plot_height=250, logo = None)
S.line(np.linspace(0,T,num_samples),Tx)
show(S)
#help(S)
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')


time taken:  0.5951652780058794 s


##### Example channel charatersitics

In [15]:
#h,r,d1,d2 = map(float,input("Channel Characteristics: Height, Range, d1 and d2 (meters): ").split())
#given channel characteristics:
h,r,d1,d2 = [100,200,9,10] # all in meters
#eigen = int(input("Number of eigenpath: "))
eigen = 4
distance = 0

##### Result for Impulse Response

Planning to use model physical model of Ray Tracing for the channel

In [35]:
## tic=timeit.default_timer()
Hn = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d2,fc)

I = figure(title="Ideal Channel Impulse Response", x_axis_label = "Delay Spread(s)", y_axis_label = "Arbritrary Units",plot_width=500, plot_height=250, logo = None)
I.line(np.linspace(0,len(Hn)/samplefreq,len(Hn)),Hn, line_color="blue")
show(I)
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')

time taken:  761.0617951609893 s


##### Received Signal
Correlating the Received Signal with Input Signal (only if it is a large Time Bandwith product) will result to an estimation of Channel Impulse Response. Examples of large Time Bandwith product are LFM and M-sequence serires. A pure tonal singal will have small Time Bandwith product because it only spans a single frequency, but very long time.

In [5]:
Rx, max_value = convolutionForReceivedSignal(Tx,Hn,Filter = True)

x_axis = np.linspace(0,len(Rx)/samplefreq,len(Rx))
print(max_value)

R = figure(title="Pure Recieved Signal", x_axis_label = "Time(s)", y_axis_label = "Ampltidue",plot_width=500, plot_height=250, logo = None)
R.line(x_axis,Rx, line_color = "black")
show(R)

0.00632624721118


###### FFT of the Impulse Response

In [6]:
Hn1 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d2)

max_new_list = np.abs(np.fft.fft(Hn1))
signalSize = Hn1.size
freq_l = np.fft.fftfreq(signalSize)
print(len(freq_l))
print(max(freq_l))
print(freq_l)

FF = figure(title="FFT of the Impulse Response Channel", x_axis_label = "Frequency(Hz)", y_axis_label = "Ampltidue",plot_width=500, plot_height=250, logo = None)
FF.line(freq_l,max_new_list)
show(FF)

#print(freq_list)

2321
0.499784575614
[ 0.          0.00043085  0.0008617  ..., -0.00129255 -0.0008617
 -0.00043085]


Changing form my code to FFT, actually yields the same result. However, it is important to use np.fft.fftfreq funciton to find out the frequency range we are working with.

In [7]:
freq_list = np.arange(0,(((samplefreq/2)/(10**3))),(((samplefreq/2)/(10**3)))/256)
Hn1 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d2)

multiple_max_new_list = []
multiple_freq_list1 = []

max_new_list1 = np.abs(np.fft.rfft(Hn1))
A = [max_value-np.mean(max_new_list1) for max_value in max_new_list1]
max_new_list2 = channelFrequencyResponse(Hn1,num_samples,T,freq_list)
B = [max_value-np.mean(max_new_list1) for max_value in max_new_list2]

signalSize = Hn1.size
freq_l = np.fft.fftfreq(signalSize,1000/samplefreq)
freq_list1=[freq for freq in freq_l if freq>=0]

multiple_max_new_list.append(A)
multiple_freq_list1.append(freq_list1)
multiple_max_new_list.append(B)
multiple_freq_list1.append(freq_list)
#multiple_max_new_list.append(max_new_list1)
#multiple_max_new_list.append(max_new_list2)

data1 = {'xs': multiple_freq_list1,
        'ys': multiple_max_new_list,
        'labels': ['w/ fft','w/o fft'],
        'mypalette': Spectral11[0:2]}

source = ColumnDataSource(data1)


FF = figure(x_axis_label = "Frequency(Hz)", y_axis_label = "Magnitude Freq Response",plot_width=800, plot_height=350, logo = None)
FF.multi_line(xs='xs', ys='ys', legend='labels', line_color = 'mypalette',source=source)
show(FF)

# Channel Frequency Response

The idea of Channel Frequency Response is that the FFT of a given channel is found out by feeding an infinitely long sinusoid (pure tonal), and convolute with the channel response. The amplitude of the recieved signal will be then be plotted agains the carrier frequency. That will produce a frequency response, and it is called the First Principle. 

Also, it is observed that locating the receiver right at the surface will yield a very small amount of amplitude, range(0.25-0.75).

Below is an example of one realization of a given Channel at:

In [8]:
#Given D2 values of the channel: 
D2_start,D2_end,D2_res = 5,25,2
D2_test = np.linspace(D2_start,D2_end,D2_res) #1cm increment
fc, bw = 5,5

#freq_list = np.arange(0,2.5,(2.5/256))

freq_final_list, max_final_new_list, freq_list = FrequencyResponse(h,r,d1,D2_test,eigen,samplefreq,fc,bw,Type = 'multi')

                                                      
roundD2_test = ['%.2f'% elem for elem in D2_test]
data = {'xs': freq_final_list,
        'ys': max_final_new_list,
        'labels': roundD2_test,
        'mypalette': viridis(len(D2_test))}#Category10[0:len(D2_test)]}

source = ColumnDataSource(data)

FI = figure(x_axis_label = "Frequency(kHz)", y_axis_label = "Ampltidue",plot_width=950, plot_height=400,logo = None)
    #FI.line(freq_finallist,max_finalnew_list)
FI.title.text = 'Frequency Response of the Channel of range {0} m'.format(r)
FI.title.align = "center"
FI.title.text_color = "black"
FI.title.text_font_size = "15px"

FI.multi_line(xs='xs', ys='ys', legend='labels', line_color = 'mypalette',source=source)
#print(max_final_new_list)
#print(freq_final_list)
show(FI)

In [9]:
#Given D2 values of the channel: 
D21,D22 = 5,9
D2_test = np.linspace(D21,D22,2) #1cm increment
fc, bw = 5,0.5

freq_final_list, max_final_new_list, CMNV, CMNS = FrequencyResponse(h,r,d1,D2_test,eigen,samplefreq,fc,bw,Type = 'duo')

                                                      
roundD2_test = ['%.2f'% elem for elem in D2_test]
data = {'xs': freq_final_list,
        'ys': max_final_new_list,
        'labels': roundD2_test,
        'mypalette': viridis(len(D2_test))}

source = ColumnDataSource(data)

FI = figure(x_axis_label = "Frequency(kHz)", y_axis_label = "Ampltidue",plot_width=950, plot_height=400,logo = None)
FI.title.text = 'Frequency Response of the Channel of range {0} m'.format(r)
FI.title.align = "center"
FI.title.text_color = "black"
FI.title.text_font_size = "15px"

FI.multi_line(xs='xs', ys='ys', legend='labels', line_color = 'mypalette',source=source)
print("point to point:", CMNV)
print("sum of squares:", CMNS)
show(FI)

point to point: 0.484873981784
sum of squares: 1.0


In [10]:
#Given D2 values of the channel: 
D21,D22 = 5,15
D2_test = np.linspace(D21,D22,2) #1cm increment
fc, bw = 5,0.5

freq_final_list, max_final_new_list, CMNV, CMNS = FrequencyResponse(h,r,d1,D2_test,eigen,samplefreq,fc,bw,Type = 'duo')

                                                      
roundD2_test = ['%.2f'% elem for elem in D2_test]
data = {'xs': freq_final_list,
        'ys': max_final_new_list,
        'labels': roundD2_test,
        'mypalette': viridis(len(D2_test))}

source = ColumnDataSource(data)

FI = figure(x_axis_label = "Frequency(kHz)", y_axis_label = "Ampltidue",plot_width=950, plot_height=400,logo = None)
FI.title.text = 'Frequency Response of the Channel of range {0} m'.format(r)
FI.title.align = "center"
FI.title.text_color = "black"
FI.title.text_font_size = "15px"

FI.multi_line(xs='xs', ys='ys', legend='labels', line_color = 'mypalette',source=source)
print("point to point:", CMNV)
print("sum of squares:", CMNS)
show(FI)

point to point: 0.711976280763
sum of squares: 1.0


In [11]:
freq_list1 = np.linspace(0.1,samplefreq/2,samplefreq/2)
wave_list  = [1500/(freq) for freq in freq_list1]
#print(wave_list)
#print(freq_final_list[1])
W = figure(x_axis_label = "Frequency(Hz)", y_axis_type="log",y_axis_label = "Wave Length",plot_width=950, plot_height=300,logo = None)
W.line(freq_list1,wave_list)
W.ray(x=[10*10**3], y=[0.02], length=50*10**3, angle=np.pi, line_width=1, line_color = '#000000')
show(W)

/Users/rickyvian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """Entry point for launching an IPython kernel.


## Processed Frequency Response
1. preamble
2. equal gains combining
3. channel estimate
4. ax+(1-a)y
5. antenna selection
6. point to point maximum value

#### Equal Gains Combining

In [12]:
EGC = np.mean(max_final_new_list,axis = 0)
freq_list = freq_list[:len(EGC)]

FIE = figure(x_axis_label = "Frequency(kHz)",y_axis_label = "Amplitude", plot_width = 700, plot_height = 250, logo = None)
FIE.title.text = "Equal Gains Combining Frequency Response at {0}m".format(r)
FIE.title.align = "center"

left, right, bottom, top = min(freq_list)-0.05, max(freq_list)+0.1, -0.05, max(EGC)+0.1
FIE.x_range=Range1d(left, right)
FIE.y_range=Range1d(bottom, top)
low_box = BoxAnnotation(top=np.mean(EGC), fill_alpha=0.05, fill_color='green')
FIE.add_layout(low_box)
FIE.line(freq_list, EGC, line_color = "black")

show(FIE)

In [13]:
D2_start,D2_end,D2_res = 0.1,10,100
D2_test2 = np.linspace(D2_start,D2_end,D2_res) #1cm increment
freq_final_list, max_final_new_list, freq_list = FrequencyResponse(h,r,d1,D2_test2,eigen,samplefreq,fc,bw)

freq_list = freq_list[:len(max_final_new_list)]

X,Y = np.meshgrid(freq_list, D2_test2)
Z = max_final_new_list

fig, ax = plt.subplots(figsize=(9, 8))

p = ax.pcolor(X, Y, Z, cmap=plt.cm.Greys, vmin=np.abs(Z).min(), vmax=np.abs(Z).max())
cb = fig.colorbar(p)
plt.title ("Frequency Response @ different receiver depth")
plt.xlabel("frequency (kHz)")
plt.ylabel('Receiver Depth')
plt.show(cb)

#X, Y = np.meshgrid(freq_list, D2_test2)
#Z = max_final_new_list
#image = Z #* 1e6

#color_mapper = LogColorMapper(palette="Viridis256", low=np.abs(Z).min(), high=np.abs(Z).max())

#plot = figure(x_range=(0,10), y_range=(0,10), toolbar_location=None)
#plot.image(image=[image], color_mapper=color_mapper,
#           dh=[1.0], dw=[1.0], x=[0], y=[0])

#color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
#                     label_standoff=12, border_line_color=None, location=(0,0))

#plot.add_layout(color_bar, 'right')

#show(plot)

NameError: name 'lowfreq' is not defined

#  Analyzing spatial diversity:
1. average data points of a certain frequency bin and compare that to deltaD2 (looking at certain bin)
2. autocorrelation (the whole frequency response)
3. Minus one signal to another signal take the maximum, to see the maximum benefits of one signal over the other.
4. averaging (But this is not good)
5. Binary Disimilrity metric. take one signal as reference, and then if it is greater in magintude = 1, if lesser = 0. and then find the yule difference / hammering difference.

## Point-to-point differential

In [ ]:
tic=timeit.default_timer()
max_final_new_list = []
freq_final_list = []

d21 = 5
Hn1 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d21)
max_new_list_1 = np.abs(np.fft.rfft(Hn1))
max_final_new_list.append(max_new_list_1)
signalSize = Hn1.size
freq_l = np.fft.fftfreq(signalSize,1000/samplefreq)
freq_list= [freq for freq in freq_l if freq>=0]
freq_final_list.append(freq_list)

d22 = 6
Hn2 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d22)
max_new_list_2 = np.abs(np.fft.rfft(Hn2))
max_final_new_list.append(max_new_list_2)
signalSize = Hn2.size
freq_l = np.fft.fftfreq(signalSize,1000/samplefreq)
freq_list= [freq for freq in freq_l if freq>=0]
freq_final_list.append(freq_list)

A=max_new_list_2
B = max_new_list_1

PTPD = [a-b for a, b in zip(A, B)]
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')

max_final_new_list.append(PTPD)
freq_final_list.append(freq_list)

#roundD2_test = ['%.2f'% elem for elem in D2_test]
#roundD2_test.append("CMNL")
label = ["d21","d22","PTPD"]
data = {'xs': freq_final_list,
        'ys': max_final_new_list,
        'labels': label,
        'mypalette': Spectral11[0:len(label)-1]+['#000000']}
#'#c70039'
source = ColumnDataSource(data)
FI = figure(x_axis_label = "Frequency(kHz)", y_axis_label = "Ampltidue",plot_width=950, plot_height=400,logo = None)
    #FI.line(freq_finallist,max_finalnew_list)
FI.title.text = 'Frequency Response of the Channel of range {0} m'.format(r)
FI.title.align = "center"
FI.title.text_color = "black"
FI.title.text_font_size = "15px"

FI.multi_line(xs='xs', ys='ys', legend='labels', line_color = 'mypalette',source=source)

show(FI)


In [ ]:
F = figure(x_axis_label = "Frequency(kHz)", y_axis_label = "PTPD values",plot_width=950, plot_height=400,logo = None)
F.line(np.linspace(0,len(PTPD),len(PTPD)),PTPD)
show(F)

## Autocorrelation method

In [ ]:
# my own function of sum of point-to-point dot product (checking time)
freq_list=np.arange(0,2.5,((2.5-0)/256))

D2_start,D2_end,D2_res = 5,10,5
d22_list = np.linspace(D2_start,D2_end,D2_res) 
CMNL = []
tic=timeit.default_timer()
d21 = 5

tic1=timeit.default_timer()
Hn1 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d21)
max_new_list_1 = channelFrequencyResponse(Hn1,num_samples,T,freq_list)
A = max_new_list_1

for d22 in d22_list:
    Hn2 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d22)
    max_new_list_2 = channelFrequencyResponse(Hn2,num_samples,T,freq_list)
    B = max_new_list_2
    corr_max_new_list = sum(a*b for a, b in zip(A, B))/(np.sqrt((sum(a**2 for a in A)*sum(b**2 for b in B))))
    CMNL.append(corr_max_new_list)


toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
print(CMNL)

FI = figure(title="xCorrelartion w.r.t 1m transmitter", x_axis_label = "Receiver Depth (m)", y_axis_label = "xcorr values",plot_width=500, plot_height=250, logo = None)
FI.line(d22_list,CMNL)
show(FI)


In [ ]:
# my own function of sum of point-to-point dot product (checking time)

D2_start,D2_end,D2_res = 50,5,100
d22_list = np.linspace(D2_start,D2_end,D2_res) 
CMNL = []
tic=timeit.default_timer()
d21 = 5

tic1=timeit.default_timer()
Hn1 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d21)
max_new_list_1 = np.abs(np.fft.fft(Hn1))
A = max_new_list_1

for d22 in d22_list:
    Hn2 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d22)
    max_new_list_2 = np.abs(np.fft.fft(Hn2))
    B = max_new_list_2
  
    corr_max_new_list = sum(a*b for a, b in zip(A, B))/(np.sqrt((sum(a**2 for a in A)*sum(b**2 for b in B))))
    CMNL.append(corr_max_new_list)


toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')

FI = figure(title="xCorrelartion w.r.t {0}m transmitter".format(d21), x_axis_label = "Receiver Depth (m)", y_axis_label = "xcorr values",plot_width=500, plot_height=250, logo = None)
FI.line(d22_list,CMNL)
show(FI)


In [ ]:
# my own function of sum of point-to-point dot product (checking time)

D2_start,D2_end,D2_res = 4,5,100
d22_list = np.linspace(D2_start,D2_end,D2_res) 
CMNL = []
tic=timeit.default_timer()
d21 = 5

tic1=timeit.default_timer()
Hn1 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d21)
max_new_list_1 = np.abs(np.fft.fft(Hn1))
A = (max_new_list_1)

for d22 in d22_list:
    Hn2 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d22)
    max_new_list_2 = np.abs(np.fft.fft(Hn2))
    B = (max_new_list_2)
  
    #corr_max_new_list = A*B/np.sqrt((A**2)*(B**2))
    corr_max_new_list = np.correlate(A,B)
    CMNL.append(corr_max_new_list)


toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
print(CMNL)
FI = figure(title="xCorrelartion w.r.t 1m transmitter", x_axis_label = "Receiver Depth (m)", y_axis_label = "xcorr values",plot_width=500, plot_height=250, logo = None)
FI.line(d22_list,CMNL)
show(FI)


In [ ]:
#find maximum of cross-correlation
#freq_list=np.arange(0,2.5,((2.5-0)/256))
d21 = 5
D2_end,D2_res = 7,500
#d22_list = np.linspace(d21,D2_end,D2_res)
#d22_list = np.linspace(0,d21,D2_res)
d22_list = np.linspace(0.1,D2_end,D2_res)
CMNL = []
tic=timeit.default_timer()
#tic1=timeit.default_timer()
Hn1 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d21)
max_new_list_1 = np.abs(np.fft.fft(Hn1)) #freq_list == samplefreq
#toc1=timeit.default_timer()
mean1 = np.mean(max_new_list_1)
B = [max_value-mean1 for max_value in max_new_list_1]
for d22 in d22_list:
    #print("time taken for corr: ", toc1-tic1, 's')
    Hn2 = ImpulseResponse(Tx,eigen,samplefreq,h,r,d1,d22)
    max_new_list_2 = np.abs(np.fft.fft(Hn2))
    mean2 = np.mean(max_new_list_2)
    A = [max_value-mean2 for max_value in max_new_list_2]
    #A = max_new_list_1
    #B = max_new_list_2
    corr_max_new_list = max(np.correlate(A,B))
    CMNL.append(corr_max_new_list)


toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
#print(CMNL)

FI = figure(title="xCorrelartion w.r.t {0}m receiver".format(d21), x_axis_label = "Receiver Depth (m)", y_axis_label = "xcorr values",plot_width=950, plot_height=550, logo = None)
FI.line(d22_list,CMNL)
show(FI)

In [ ]:
#attempt to do a 2D autocorrelation

tic=timeit.default_timer()
D2_end = 10
D2_res = 100
multipleCMNL = []
lowfreq, highfreq = 0,10
d21_list = np.linspace(0.1,D2_end,D2_res)
for i in range(len(d21_list)):
    d22_list = np.linspace(0.1,D2_end,D2_res) 
    CMNL = CorrDiversity(Tx,h,r,d1,d21_list[i],d22_list,eigen,samplefreq,lowfreq, highfreq)
    multipleCMNL.append(CMNL)

X,Y = np.meshgrid(d22_list,d21_list)
Z = multipleCMNL

image = Z
color_mapper = LogColorMapper(palette="Viridis256", low=np.abs(Z).min(), high=np.abs(Z).max())

plot = figure(x_range=(0,np.abs(X).max()), y_range=(0,np.abs(Y).max()), title="XCorr of two receivers at different depths", x_axis_label = "different D22", y_axis_label = "D21 reference point")
plot.image(image=[image], color_mapper=color_mapper,
           dh=[np.abs(Y).max()], dw=[np.abs(X).max()], x=[0], y=[0], name = 'Xcorr values')

color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

plot.add_layout(color_bar, 'right')
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
show(plot)

In [ ]:
#sum of point-to-point dot product (cross correlation at zero frequency delay)
#freq_list=np.arange(0,2.5,((2.5-0)/256))
H,R,D1,Eigen = 100, 100, 7, 4
delta = 5
fc,bw = 5, 4 #khz

d21 = 5
d22_list = np.linspace(d21-delta,d21+delta,50) 

CMNL = CorrDiversity(Tx,H,R,D1,d21,d22_list,Eigen,samplefreq, fc, bw)

FI = figure(title="xCorrelartion of {0}m reciever at {1}m delta".format(d21,delta), x_axis_label = "Receiver Depth (m)", y_axis_label = "xcorr values",plot_width=950, plot_height=550, logo = None)
FI.line(d22_list,CMNL)
show(FI)

In [ ]:
#sum of point-to-point dot product (cross correlation at zero frequency delay)
#freq_list=np.arange(0,2.5,((2.5-0)/256))
H,R,D1,Eigen = 100, 200, 9, 4
fc,bw = 5, 0.5 #khz

d21 = 5
d22 = 9

CMNV = CorrDiversity(Tx,H,R,D1,d21,d22,Eigen,samplefreq, fc, bw, Type = 'single')
print("xcorr value:", CMNV)

In [ ]:
#sum of point-to-point dot product (cross correlation at zero frequency delay)
#freq_list=np.arange(0,2.5,((2.5-0)/256))
H,R,D1,Eigen = 100, 200, 9, 4
fc,bw = 5, 0.5 #khz

d21 = 5
d22 = 15

CMNV = CorrDiversity(Tx,H,R,D1,d21,d22,Eigen,samplefreq, fc, bw, Type = 'single')
print("xcorr value:", CMNV)

## Further Analysis on the autocorrelation metric

Bandwith(OK)
Dmin-Dmax = len array(OK)
depth of array (OK)
H,R,D1 (OK) 

In [ ]:
#sum of point-to-point dot product (cross correlation at zero frequency delay)
#freq_list=np.arange(0,2.5,((2.5-0)/256))
tic = timeit.default_timer()
H,R,D1,Eigen = 100, 100, 9, 4
D2min, D2max = 5,7
deltaMin, deltaMax = 0,50
thres = 0.5
fc, bw = 5,0.5

delta_list, multiAvgCMNL, imptDelta = avgXCorrForDeltas(Tx, H, R, D1, D2min, D2max,Eigen, deltaMin, deltaMax, thres, fc, bw, Len = 'valid')

FI = figure(title="average xCorrelartion for different deltas", x_axis_label = "Delta (m)", y_axis_label = "xcorr values",plot_width=950, plot_height=500, logo = None)
FI.line(delta_list,multiAvgCMNL)
FI.ray(x=[deltaMax], y=[thres], length=deltaMax, angle=np.pi, line_width=1, line_color = '#000000')
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
print("delta point:", imptDelta)
show(FI)

In [ ]:
#sum of point-to-point dot product (cross correlation at zero frequency delay)
#freq_list=np.arange(0,2.5,((2.5-0)/256))
tic = timeit.default_timer()
H,R,D1,Eigen = 500, 100, 9, 4
D2min,lenReceiverArray = 5, 2
deltaMin, deltaMax = 0,50
thres = 0.5
fc, bw = 5,0.5

D2max = lenReceiverArray+D2min

delta_list, multiAvgCMNL, imptDelta = avgXCorrForDeltas(Tx, H, R, D1, D2min, D2max,Eigen, deltaMin, deltaMax, thres, fc, bw)

FI = figure(title="average xCorrelartion for different deltas", x_axis_label = "Delta (m)", y_axis_label = "xcorr values",plot_width=950, plot_height=500, logo = None)
FI.line(delta_list,multiAvgCMNL)
FI.ray(x=[deltaMax], y=[thres], length=deltaMax, angle=np.pi, line_width=1, line_color = '#000000')
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
print("delta point:", imptDelta)
show(FI)

## Observing how deltas changes with different variables.

In [ ]:
#freq_list=np.arange(0,2.5,((2.5-0)/256))
tic = timeit.default_timer()
H,D1,Eigen = 100, 9, 4
D2min,lenReceiverArray = 5, 2
deltaMin, deltaMax = 0,10
thres = 0.5
fc, bw = 5,0.5

D2max = 2*lenReceiverArray+D2min

R_list = np.linspace(1,100,100)
imptDelta_list = []

for R in R_list:
    delta_list, multiAvgCMNL, imptDelta = avgXCorrForDeltas(Tx, H, R, D1, D2min, D2max, Eigen, deltaMin, deltaMax, thres,fc,bw)
    imptDelta_list.append(imptDelta)

FI = figure(title="Effect of Range in impt Delta", x_axis_label = "Range (m)", y_axis_label = "impt Delta(m)",plot_width=950, plot_height=550, logo = None)
FI.line(R_list,imptDelta_list)
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
#print("delta point:", imptDelta)
show(FI)

In [ ]:
tic = timeit.default_timer()
R,D1,Eigen = 100, 9, 4
D2min,lenReceiverArray = 5, 2
deltaMin, deltaMax = 0,10
thres = 0.5
fc, bw = 5,0.5

D2max = lenReceiverArray+D2min

H_list = np.linspace(100,500,50)
imptDelta_list = []
for H in H_list:
    delta_list, multiAvgCMNL, imptDelta = avgXCorrForDeltas(Tx, H, R, D1, D2min, D2max, Eigen, deltaMin, deltaMax, thres,lowfreq,highfreq)
    imptDelta_list.append(imptDelta)

FI = figure(title="Effect of Height in impt Delta", x_axis_label = "Height (m)", y_axis_label = "impt Delta(m)",plot_width=950, plot_height=550, logo = None)
FI.line(H_list,imptDelta_list)
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
#print("delta point:", imptDelta)
show(FI)

In [ ]:
tic = timeit.default_timer()
H,R,D1,Eigen = 100, 100, 9, 4
lenReceiverArray = 2
deltaMin, deltaMax = 0,10
thres = 0.5
fc, bw = 5,0.5


D2min_list = np.linspace(5,50,50)
imptDelta_list = []
for D2min in D2min_list:
    D2max = 2*lenReceiverArray+D2min
    delta_list, multiAvgCMNL, imptDelta = avgXCorrForDeltas(Tx, H, R, D1, D2min, D2max, Eigen, deltaMin, deltaMax, thres,lowfreq,highfreq)
    imptDelta_list.append(imptDelta)

FI = figure(title="Effect of Receiver depth on impt Delta", x_axis_label = "Receiver Depth (m)", y_axis_label = "impt Delta(m)",plot_width=950, plot_height=550, logo = None)
FI.line(D2min_list,imptDelta_list)
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
#print("delta point:", imptDelta)
show(FI)

In [ ]:
tic = timeit.default_timer()
H,R,D1,Eigen = 100,100, 9, 4
D2min,lenReceiverArray = 5, 2
deltaMin, deltaMax = 0,10
thres = 0.5
bw = 0.5 #kHz

D2max = 2*lenReceiverArray+D2min

fc_list = np.linspace(1,9,50)
imptDelta_list = []
for fc in fc_list:
    delta_list, multiAvgCMNL, imptDelta = avgXCorrForDeltas(Tx, H, R, D1, D2min, D2max, Eigen, deltaMin, deltaMax, thres,fc,bw)
    imptDelta_list.append(imptDelta)

FI = figure(title="Effect of Center Frequency on impt Delta", x_axis_label = "Center Frequency (kHz)", y_axis_label = "impt Delta(m)",plot_width=950, plot_height=550, logo = None)
FI.line(fc_list,imptDelta_list)
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
#print("delta point:", imptDelta)
show(FI)

In [ ]:
ic = timeit.default_timer()
H,R,D1,Eigen = 100,100, 9, 4
D2min,lenReceiverArray = 0, 5
deltaMin, deltaMax = 0,10
thres = 0.5
fc = 5 #kHz

D2max = 2*lenReceiverArray+D2min

bw_list = np.linspace(0.5,10,50)
imptDelta_list = []
for bw in bw_list:
    delta_list, multiAvgCMNL, imptDelta = avgXCorrForDeltas(Tx, H, R, D1, D2min, D2max, Eigen, deltaMin, deltaMax, thres,fc,bw)
    imptDelta_list.append(imptDelta)

FI = figure(title="Effect of Bandwith on impt Delta", x_axis_label = "Bandwith (kHz)", y_axis_label = "impt Delta(m)",plot_width=950, plot_height=550, logo = None)
FI.line(bw_list,imptDelta_list)
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
#print("delta point:", imptDelta)
show(FI)

# Random Observations

###### Range vs Delay Spread
Delay Spread are usually caused by delay from the vertical distance of the channel.

It is observed that as the distance between the transmitter and reciever increase, the delay spread decreases exponentially. This is geometrically sounds, as the wider the channel, the eigen path needs to travel less vertical distance, hence less delay spread. 

In [ ]:
i = 0
r_list = []
tao_list = []
samplefreq = 5.0*10**3 #Hz
freq = 2
Tx = CWsinewave(freq,T,num_samples)
Height,Dist1,Dist2 = 100,7,5
Range = 1
while Range < 4000: 
# Ray Modelling (non-recursive)
    ImpulseResponse(Tx,eigen,samplefreq,Height,Range,Dist1,Dist2)
    r_list.append(int(Range))
    #print(r_list)
    tao_list.append(tao)
    #print(tao_list)
    Range+=5

RD = figure(title="Range vs Delay Spread", x_axis_label = "range of the transmitter and receiver (m)", y_axis_label = "Delay Spread of the last eigen path (s)",plot_width=600, plot_height=300, logo = None)
RD.line(r_list,tao_list)
show(RD)

In [ ]:
i = 0
h_list = []
tao_list = []
samplefreq = 5.0*10**3 #Hz
freq = 2
Tx = CWsinewave(freq,T,num_samples)
Range,Dist1,Dist2 = 100,7,5
Height = 1
while Range < 500: 
# Ray Modelling (non-recursive)
    ImpulseResponse(Tx,eigen,samplefreq,Height,Range,Dist1,Dist2)
    h_list.append(int(Height))
    #print(r_list)
    tao_list.append(tao)
    #print(tao_list)
    Height+=5

HD = figure(title="Height vs Delay Spread", x_axis_label = "height of the channel (m)", y_axis_label = "Delay Spread of the last eigen path (s)",plot_width=600, plot_height=300, logo = None)
HD.line(h_list,tao_list)
show(HD)

In [ ]:
#Ray Modelling
split = 0
e,s,b = 0,0,0
tr = [0,r]
x = tr
dd = [d1,d2]
y = [d1,d2]
yi = [d1,d2]

X = []
Y = []

for j in range (int(eigen/3)+1):
    for i in range (3):
        if eigen == e:
            break
        split += 1
        
        if s==0 and b ==0:
            split = 0
            
        if s<b:
            split -= 1
        x = np.linspace(0,r,split+2)
        
        if s>b:
            y.insert(len(y)-1,h)
            yi.insert(len(yi)-1,0)
            
        if s == b and s!=0:
            y.insert(len(y)-1,0)
            yi.insert(len(yi)-1,h)
            
        X.append(x)
        Y.append(y)
        
        print("START POINT")
        print("s,b:", s,b)
        print ("split: ", split)
        print ("x points:   ", X)
        print("y points: ", Y)
        print("yinvert points: ", yi)
        print("END POINT")
        print(dd, tr)
# to change the s and b values
        if i == 0 or i == 2:
            s+= 1
        elif i == 1:
            s-= 1
            b+= 1

RM = figure(title = "Ray Modelling", x_axis_label = "range (m)", y_axis_label = "depth (m)",plot_width = 500, plot_height = 500, logo = None,toolbar_location = None)
RM.xgrid.grid_line_color = None
RM.ygrid.grid_line_color = None
left, right, bottom, top = 0-5, r+5, h, 0
RM.x_range=Range1d(left, right)
RM.y_range=Range1d(bottom, top)
numlines=eigen
mypalette=Spectral11[0:numlines]
RM.multi_line(X,Y,line_color=mypalette,line_width=0.5)
RM.asterisk(tr,dd, size=20, color="#F0027F")
show(RM)

given the length of the square is $(x^2)$, what is the area of it?

In [ ]:
# BOKEH PLOTTING LAYOUT

#low_box = BoxAnnotation(top=80, fill_alpha=0.1, fill_color='red')
#mid_box = BoxAnnotation(bottom=80, top=180, fill_alpha=0.1, fill_color='green')
#high_box = BoxAnnotation(bottom=180, fill_alpha=0.1, fill_color='red')

#FI.add_layout(low_box)
#FI.add_layout(mid_box)
#FI.add_layout(high_box)


#cr = FI.circle(freq_final_list,max_final_new_list, size=20,
#                fill_color="grey", hover_fill_color="firebrick",
#                fill_alpha=0.05, hover_alpha=0.3,
#                line_color=None, hover_line_color="white")

#FI.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))